# NoSQL (HBase) (sesión 5)

![Image of HBase](http://hbase.apache.org/images/hbase_logo_with_orca_large.png)

Esta hoja muestra cómo acceder a bases de datos MongoDB y también a conectar la salida con Jupyter. Se puede utilizar el *shell* propio de MongoDB en la máquina virtual usando el programa `mongo`. La diferencia es que ese programa espera código Javascript y aquí trabajaremos con Python.

Lo primero que se hará será instalar la versión 3.2 de mongodb. Ésta no viene por defecto en la máquina virtual, y sólo hay que hacerlo una vez, aunque no pasa nada si se repite.

In [ ]:
# Config
%env DIR=/vagrant

In [1]:
!~/start-hbase.sh

Starting namenodes on [localhost]
localhost: namenode running as process 2081. Stop it first.
localhost: datanode running as process 2230. Stop it first.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: secondarynamenode running as process 2409. Stop it first.
starting yarn daemons
resourcemanager running as process 2549. Stop it first.
localhost: nodemanager running as process 2675. Stop it first.
localhost: zookeeper running as process 3198. Stop it first.
master running as process 3287. Stop it first.
regionserver running as process 3409. Stop it first.
thrift running as process 3515. Stop it first.


In [ ]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [5]:
import happybase

host = '127.0.0.1'
connection = happybase.Connection(host)
connection.tables()

[]

 - Format: 7zipped
 - Files:
   - **badges**.xml
       - UserId, e.g.: "420"
       - Name, e.g.: "Teacher"
       - Date, e.g.: "2008-09-15T08:55:03.923"
   - **comments**.xml
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **posts**.xml
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       - CommentCount
       - FavoriteCount
   - **posthistory**.xml
	   - Id
	   - PostHistoryTypeId
			- 1: Initial Title - The first title a question is asked with.
			- 2: Initial Body - The first raw body text a post is submitted with.
			- 3: Initial Tags - The first tags a question is asked with.
			- 4: Edit Title - A question's title has been changed.
			- 5: Edit Body - A post's body has been changed, the raw text is stored here as markdown.
			- 6: Edit Tags - A question's tags have been changed.
			- 7: Rollback Title - A question's title has reverted to a previous version.
			- 8: Rollback Body - A post's body has reverted to a previous version - the raw text is stored here.
			- 9: Rollback Tags - A question's tags have reverted to a previous version.
			- 10: Post Closed - A post was voted to be closed.
			- 11: Post Reopened - A post was voted to be reopened.
			- 12: Post Deleted - A post was voted to be removed.
			- 13: Post Undeleted - A post was voted to be restored.
			- 14: Post Locked - A post was locked by a moderator.
			- 15: Post Unlocked - A post was unlocked by a moderator.
			- 16: Community Owned - A post has become community owned.
			- 17: Post Migrated - A post was migrated.
			- 18: Question Merged - A question has had another, deleted question merged into itself.
			- 19: Question Protected - A question was protected by a moderator
			- 20: Question Unprotected - A question was unprotected by a moderator
			- 21: Post Disassociated - An admin removes the OwnerUserId from a post.
			- 22: Question Unmerged - A previously merged question has had its answers and votes restored.
		- PostId
		- RevisionGUID: At times more than one type of history record can be recorded by a single action.  All of these will be grouped using the same RevisionGUID
		- CreationDate: "2009-03-05T22:28:34.823"
		- UserId
		- UserDisplayName: populated if a user has been removed and no longer referenced by user Id
		- Comment: This field will contain the comment made by the user who edited a post
		- Text: A raw version of the new value for a given revision
			- If PostHistoryTypeId = 10, 11, 12, 13, 14, or 15  this column will contain a JSON encoded string with all users who have voted for the PostHistoryTypeId
			- If PostHistoryTypeId = 17 this column will contain migration details of either "from <url>" or "to <url>"
		- CloseReasonId
			- 1: Exact Duplicate - This question covers exactly the same ground as earlier questions on this topic; its answers may be merged with another identical question.
			- 2: off-topic
			- 3: subjective
			- 4: not a real question
			- 7: too localized
   - **postlinks**.xml
     - Id
     - CreationDate
     - PostId
     - RelatedPostId
     - PostLinkTypeId
       - 1: Linked
       - 3: Duplicate
   - **users**.xml
     - Id
     - Reputation
     - CreationDate
     - DisplayName
     - EmailHash
     - LastAccessDate
     - WebsiteUrl
     - Location
     - Age
     - AboutMe
     - Views
     - UpVotes
     - DownVotes
   - **votes**.xml
     - Id
     - PostId
     - VoteTypeId
        - ` 1`: AcceptedByOriginator
        - ` 2`: UpMod
        - ` 3`: DownMod
        - ` 4`: Offensive
        - ` 5`: Favorite - if VoteTypeId = 5 UserId will be populated
        - ` 6`: Close
        - ` 7`: Reopen
        - ` 8`: BountyStart
        - ` 9`: BountyClose
        - `10`: Deletion
        - `11`: Undeletion
        - `12`: Spam
        - `13`: InformModerator
     - CreationDate
     - UserId (only for VoteTypeId 5)
     - BountyAmount (only for VoteTypeId 9)

In [ ]:
%%bash
(test -e $DIR/Posts.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Posts.csv.gz -O - 2>/dev/null | gunzip > $DIR/Posts.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Users.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Users.csv.gz -O - 2>/dev/null | gunzip > $DIR/Users.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Tags.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Tags.csv.gz -O - 2>/dev/null | gunzip > $DIR/Tags.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Comments.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Comments.csv.gz -O - 2>/dev/null | gunzip > $DIR/Comments.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e $DIR/Votes.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Votes.csv.gz -O - 2>/dev/null | gunzip > $DIR/Votes.csv) \
  && echo OK)

In [ ]:
%%bash
mongoimport --db stackoverflow --collection posts --drop --type csv \
   --headerline --host=127.0.0.1:$MONGOPORT --file $DIR/Posts.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection users --drop --type csv \
   --headerline --host=127.0.0.1:$MONGOPORT --file $DIR/Users.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection votes --drop --type csv \
   --headerline --host=127.0.0.1:$MONGOPORT --file $DIR/Votes.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection comments --drop --type csv \
   --headerline --host=127.0.0.1:$MONGOPORT --file $DIR/Comments.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection tags --drop --type csv \
   --headerline --host=127.0.0.1:$MONGOPORT --file $DIR/Tags.csv

## EJERCICIO: Con HBase, mostrar las consultas RQ1 a RQ4 de la sesión 2